In [ ]:
"""Суть задачи 🧬
У нас есть ДНК-последовательность, состоящая всего из 4 букв: A, C, G, T. 
В Python строка хранит каждый символ как Unicode, что занимает 8 байт = 64 бита на символ! 
Это дикая неэффективность, потому что для кодирования 4 вариантов достаточно всего 2 бит.

Математика эффективности:
4 варианта = 2² → нужно 2 бита

Обычная строка: 8 байт = 64 бита на символ

Сжатый вариант: 2 бита на символ

Экономия: (64-2)/64 × 100% = 96.875% (в книге считают 8 бит → 75%)
"""